<a href="https://colab.research.google.com/github/MarcosRolando/OrgaDeDatosTP2/blob/main/neuralNetwoork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import math as mt
import regex as re
from tensorflow import feature_column


In [ ]:
df = pd.read_csv('/content/Test_TP2_Datos_2020-2C.csv')
df[df['Territory'] == 'Vietnam']

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,Billing_Country,Account_Name,Opportunity_Name,Opportunity_ID,Sales_Contract_No,Account_Owner,Opportunity_Owner,Account_Type,Opportunity_Type,Quote_Type,Delivery_Terms,Opportunity_Created_Date,Brand,Product_Type,Size,Product_Category_B,Price,Currency,Last_Activity,Quote_Expiry_Date,Last_Modified_Date,Last_Modified_By,Product_Family,Product_Name,ASP_Currency,ASP,ASP_(converted)_Currency,ASP_(converted),Planned_Delivery_Start_Date,Planned_Delivery_End_Date,Month,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Prod_Category_A


In [ ]:
df = pd.read_csv('/content/Train_TP2_Datos_2020-2C.csv')

#df = df[df['Brand'] == 'No']
'''
df['days'] = (df['Planned_Delivery_End_Date'] - df['Planned_Delivery_Start_Date']).dt.days
df['days'] = df.groupby('Opportunity_ID')['days'].transform('max')
df['days'] = df['days'].replace({np.nan:10.0}) #Reemplazo con 10 porque los que no tienen fecha final ganan el 60%, y el analisis de los datos da que el 60% es maso a los 10 dias. Asi no jodo el resto de los datos
df['days'] = df['days'] > 0

df['TRF'] = df.groupby('Opportunity_ID')['TRF'].transform('max')

df = df[(df['Has_Contract_Number'] == 'Yes')]
df['Ratio'] = df.groupby('Territory')['Stage'].transform('mean')
df[df['Ratio'] < 0.8]['Territory'].value_counts()'''
#df[df['Products'] == 'Product_Name_111']
df['Product_Family'].value_counts()


Product_Family_77     1345
Product_Family_133    1249
Product_Family_132     763
Product_Family_212     636
Product_Family_100     619
                      ... 
Product_Family_241       1
Product_Family_51        1
Product_Family_40        1
Product_Family_34        1
Product_Family_79        1
Name: Product_Family, Length: 227, dtype: int64

Tuneamos algunas cosas del DataFrame que notamos en el analisis de los datos.

In [ ]:
def preprocess_dataframe(df):

  df.fillna(value=0, inplace=True) #Reemplazamos NAN por 0, ya que NAN rompe a Tensorflow

  #Renombramos las columnas que tienen caracteres que TensorFlow no acepta como validos.
  #Estos particularmente son whitespace, coma y parentesis por ejemplo.
  df.rename(columns={'ASP_(converted)':'ASP_converted','Pricing, Delivery_Terms_Quote_Appr':
                    'Pricing_Delivery_Terms_Quote_Appr','Pricing, Delivery_Terms_Approved':
                    'Pricing_Delivery_Terms_Approved','Source ':'Source'},inplace=True)

  df = df[df['Stage'].isin(['Closed Won', 'Closed Lost'])]
  df.loc[:, 'Stage'].replace({'Closed Won':1, 'Closed Lost':0}, inplace=True) #0 corresponde a que el caso fue Closed Lost, 1 a que fue Closed Won. Asi tenemos un problema de clasificacion binario que puede entender la red neuronal.

  df.loc[:, 'Planned_Delivery_Start_Date'] = pd.to_datetime(df['Planned_Delivery_Start_Date'], 'coerce',
                                                                  format='%m/%d/%Y')
  df.loc[:, 'Planned_Delivery_End_Date'] = pd.to_datetime(df['Planned_Delivery_End_Date'], 'coerce',
                                                                                      format='%m/%d/%Y')
  df = df[df['Opportunity_ID'] != 9773] #Hardcodeo este filtrado porque el id 9773 tiene mal cargada la fecha de delivery end, dando una diferencia de 200 anios xd"

  #Pongo .loc porque pandas me jode con warnings que son falsos positivos de slice copy"
  #Gracias Pandas!"

  #Creamos una nueva columna (Feature Engineering) que contiene la longitud en dias 
  #estimada de la operacion. En el informe habiamos encontrado que aparentaba haber
  #una relacion cuadratica de decrecimiento a medida que aumentaban los dias donde disminuia
  #la chance de completar la operacion.
  df['Delta_Time'] = df['Planned_Delivery_End_Date'] - df['Planned_Delivery_Start_Date']
  df.loc[:, 'Delta_Time'] = df['Delta_Time'].dt.days
  df['Delta_Time'] = df['Delta_Time'].replace({np.nan:10.0}) #Reemplazo con 10 porque los que no tienen fecha final ganan el 60%, y el analisis de los datos da que el 60% es maso a los 10 dias. Asi no jodo el resto de los datos
  df['Delta_Time'] = df.groupby('Opportunity_ID')['Delta_Time'].transform('max')

  #Pasamos todo a dolares
  currency_conversion = {'AUD':0.707612, 'EUR':1.131064, 'GBP':1.318055, 'JPY':0.008987, 'USD':1.0}
  df['Total_Taxable_Amount_Currency'] = df[['Total_Taxable_Amount_Currency']].replace(currency_conversion)
  df['Total_Taxable_Amount'] = df['Total_Taxable_Amount_Currency'] * df['Total_Taxable_Amount']

  #Modifico la columna Brand para que en vez de decir que marca es, solo diga
  #si tiene o no marca. Es importante aclarar que verificamos que siempre que una oportunidad
  #tiene un producto con marca entonces todos sus productos tienen marca. Esto se cumple
  #tanto en el set de entrenamiento como en el de test, por lo tanto al hacer drop_duplicates
  #no nos va a pasar nunca el caso donde nos pudieramos quedar con una entrada de producto
  #sin marca mientras que algun otro producto si tuviera, ya que confirmamos que o todos tienen
  #marca o ninguno tiene.
  df.loc[df['Brand'] == 'None', 'Brand'] = 'No'
  df.loc[df['Brand'] != 'No', 'Brand'] = 'Yes'

  #Agrego una columna que indica si tiene o no numero de contrato
  df.loc[:, 'Sales_Contract_No'][df['Sales_Contract_No'] != 'None'] = 'Yes'
  df.loc[:, 'Sales_Contract_No'][df['Sales_Contract_No'] == 'None'] = 'No'
  df.rename(columns={'Sales_Contract_No':'Has_Contract_Number'}, inplace=True)

  #Agrego una columna que indique la cantidad de productos que tiene esa
  #oportunidad
  df['Product'] = 1
  df['Product_Amount'] = df.groupby('Opportunity_ID')['Product'].transform(lambda x: x.sum())

  #Agrego una columna que indica si el owner de la cuenta es el mismo que el de la oportunidad
  #o no
  df['Same_Owner'] = (df['Account_Owner'] == df['Opportunity_Owner'])
  df['Same_Owner'] = df['Same_Owner'].replace({False:'No', True:'Yes'})

  #Agrego una columna que indica si tiene o no fecha de expiracion
  df['Quote_Expiry_Date'] = (df['Quote_Expiry_Date'] != 'NaT')
  df.rename(columns={'Quote_Expiry_Date':'Has_Expiry_Date'}, inplace=True)
  df['Has_Expiry_Date'] = df['Has_Expiry_Date'].replace({True:'Yes',False:'No'})

  #Reemplazo las 4 columnas de aprobacion por solo 2 columnas que indiquen si tuvo la aprobacion
  #de delivery y burocratica o no. Recalco que si nunca la necesito seria equivalente a si
  #la necesito y la consiguio.
  df['Delivery_Approved'] = df['Pricing_Delivery_Terms_Quote_Appr'] + df['Pricing_Delivery_Terms_Approved']
  df['Delivery_Approved'] = df['Delivery_Approved'].replace({0:1, 1:0, 2:1})
  df['Bureaucratic_Code_Approved'] = df['Bureaucratic_Code_0_Approval'] + df['Bureaucratic_Code_0_Approved']
  df['Bureaucratic_Code_Approved'] = df['Bureaucratic_Code_Approved'].replace({0:1, 1:0, 2:1})
  df['Approved'] = df['Delivery_Approved'] & df['Bureaucratic_Code_Approved']

  #Cambio TRF por una columna que es el valor medio de los TRF de la oportunidad
  df["TRF"] = df.groupby("Opportunity_ID")["TRF"].transform("mean")

  def combineProducts(x):
    products = ""
    added = []
    for product in x:
      product = re.findall('\d+', product)[0]
      if added.count(product) == 0:
        products += (product)
        added.append(product)
    return products

  #Junto todos los productos en una sola entrada
  df['Products'] = df.groupby('Opportunity_ID')['Product_Family'].transform(combineProducts)

  #Pruebo volar duplicados, solo cambia el producto. Si el producto no importa
  #entonces volar duplicados no deberia importar. Obviamente vuelo el producto en el que
  #quede tambien.
  df.drop_duplicates('Opportunity_Name',inplace=True)
  df.drop(columns=['Product_Name','Product_Family','Opportunity_Name'],inplace=True)

  #Normalizo las columnas numericas
  normalized_columns = ['ASP_converted','TRF','Total_Taxable_Amount', 'Product_Amount', 'Delta_Time']
  for column in normalized_columns:
    df[column] = (df[column] - df[column].mean()) / df[column].std()

  #Borro columnas que tengan el mismo dato en todas las entradas, o inconsecuentes como el ID / Opportunity_ID
  #Algunas columnas borradas son porque pienso que no tienen incidencia, ir viendo.
  #TODO: Analizar si el Sales_Contract_No no es que importe el numero en si, sino si tiene
  #o no tiene numero de contrato. Por ahora no lo meto como input.
  #TODO: Ver el mismo tema con la columna 'Price', la mayoria tiene None u Other
  #y solo unos pocos tienen precio numerico. Quiza importe que tenga precio o no tenga,
  #o si no tiene precio quiza importe si es None u Other. Por ahora no lo pongo
  #como input.
  df.drop(columns=['Submitted_for_Approval', 'Last_Activity', 'ASP_(converted)_Currency', 
                  'Prod_Category_A', 'ID', 'Opportunity_ID', 
                   'Actual_Delivery_Date'],inplace=True)

  #Drop columnas que quiza podamos usar pero por ahora no las uso
  df.drop(columns=['Account_Created_Date','Opportunity_Created_Date',
                  'Last_Modified_Date',
                  'Planned_Delivery_Start_Date','Planned_Delivery_End_Date',
                  'Month',
                   'Delivery_Year',
                  'Price','ASP','ASP_Currency','Total_Amount_Currency',
                  'Total_Amount','Total_Taxable_Amount_Currency','Currency',
                   'Product_Category_B','Last_Modified_By', 'Account_Owner',
                   'Opportunity_Owner','Account_Name','Product_Type','Size',
                   'Billing_Country', 'Pricing_Delivery_Terms_Quote_Appr',
                   'Pricing_Delivery_Terms_Approved', 'Bureaucratic_Code_0_Approval',
                   'Bureaucratic_Code_0_Approved',
                   'Total_Taxable_Amount','Product_Amount','ASP_converted',
                   'Approved','TRF','Same_Owner']
                   ,inplace=True)
  
  #Definimos que tipo de feature es cada columna

  #Debemos separar algunos de los registros para armar un set de test propio (no el de la catedra). De esta forma sabremos rapidamente
  #si nuestro modelo esta dando resultados optimos o no sin necesidad de estar subiendo el TP a Kaggle constantemente.
  #Sin embargo, no queremos usar tantos registros ya que estariamos disminuyendo el set de entrenamiento considerablemente.
  #Podemos empezar reservando 2000 registros para el test de prueba y ver que onda. Pasariamos de tener 16 mil a 14 mil 
  #registros para el set de entrenamiento, no es una perdida importantisima creo en principio, asi que arrancamos con eso.

  #Por otro lado, nuestro test de prueba deberia tener un 50 50 de Closed Won y Closed Lost, por lo que no podemos elegir asi nomas
  #al azar.

  return df

In [ ]:
# Metodo que pasa de DataFrame de Pandas a un DataSet de TensorFlow
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('Stage') #Retorna la columna Stage, eliminandolo simultaneamente del DataFrame. 'Stage' seria nuestra columna 'target', es decir, lo que queremos predecir.
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels)) #Crea un DataSet cuyos elementos son slices de los tensores pasados a la funcion. Ver documentacion oficial para comprender bien.
  #En pocas palabras, le pasamos las columnas con los datos como diccionario (estilo 'columna':[dato1,dato2,dato3]) y una lista con los resultados estilo [resu1,resu2,resu3].
  #Se genera un DataSet del estilo [('columna':[dato1,dato2,dato3], [resu1, resu2, resu3])].
  #En realidad 'columna' es una lista de todas las columnas con sus correspondientes datos, pero se entiende la idea creo.
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe)) #Al tener el buffer_size del mismo tamanio que la cantidad de datos del dataset, tenemos perfect shuffling (ver documentacion para comprender).
    #Basicamente mezlcamos el dataset para que luego los batches que se armen contengan distintos elementos si lo entrenamos distintas veces.
  ds = ds.batch(batch_size) #Arma batches de tamanio batch_size entre elementos consecutivos del DataSet.
  return ds

In [ ]:
#Arma las features, asignando las columnas del DataFrame segun corresponda al tipo de feature
#(numerico, categorico, etc). Entiendase por feature a las columnas del DataFrame.
def set_up_feature_columns(dataframe, numeric_columns, indicator_columns, bucket_columns, crossed_columns, hashed_columns):
  features = []

  #numeric features
  for column_name in numeric_columns:
    features.append(feature_column.numeric_column(column_name))

  #bucket features
  boundaries = [] #En principio este boundary es solo para la columna 'Delta Time'. Ver de como generalizar.
  for i in range(37):
    boundaries.append(i*10.0)

  for column_name in bucket_columns:
    range_column = feature_column.numeric_column(column_name)
    bukect_column = feature_column.bucketized_column(range_column, boundaries)
    features.append(bukect_column)

  #indicator features (one-hot value vector, para aquellas columnas categoricas de pocas opciones)
  for column_name in indicator_columns:
    categorical_column = feature_column.categorical_column_with_vocabulary_list(
                                          column_name, dataframe[column_name].unique())
    indicator_column = feature_column.indicator_column(categorical_column)
    features.append(indicator_column)

  #crossed features
  for crossed_feature in crossed_columns:
    categorical_columns = []
    possible_values = 1
    for column_name in crossed_feature:
      column_keys = dataframe[column_name].unique()
      possible_values = possible_values * len(column_keys)
      categorical_columns.append(feature_column.categorical_column_with_vocabulary_list(
                                            column_name, column_keys))
    crossed_feature = feature_column.crossed_column(categorical_columns, hash_bucket_size=possible_values) #Ponemos la cantidad de buckets justos para representar todas las combinaciones
    features.append(feature_column.indicator_column(crossed_feature))


  #hashed features
  for column_name in hashed_columns:
    hashed_column = feature_column.categorical_column_with_hash_bucket(
                  column_name, hash_bucket_size=100)
    features.append(feature_column.indicator_column(hashed_column))

  return features

Preparamos los features para el modelo, es decir, seteamos cada una de las columnas que vayamos a utilizar del DataFrame. Luego generamos el DataSet en base al DataFrame para darselo como input al modelo.

In [ ]:
  #Columnas que consideramos numericas
  numeric_columns = ['Delta_Time']#['Total_Taxable_Amount','Product_Amount','ASP_converted','TRF']

  #Columnas que consideramos clasificatorias con rango numerico
  bucket_columns = []

  #Columnas que queremos hashear (no se si es LSH lo que usa esto, pruebo)
  hashed_columns = ['Products']

  #Columnas que consideramos categoricas de pocos valores posibles
  indicator_columns = []#'Region', 
                       #'Source', 
                       #'Account_Type',
                       #'Opportunity_Type']
                        #'Last_Modified_By'
                        #'Product_Family', 'Product_Name',
                        #'Account_Owner', 'Opportunity_Owner', 'Account_Name'
                        #'Territory', 'Billing_Country'

  #Columnas crossed, aquellas que queremos un parametro por combinacion
  crossed_columns = [['Brand', 'Has_Contract_Number'],#,'Has_Expiry_Date'],
                     #['Has_Contract_Number','Delivery_Terms','Delivery_Approved'],
                     #['Has_Contract_Number','Bureaucratic_Code','Bureaucratic_Code_Approved'],
                     ['Has_Contract_Number','Delivery_Quarter']]

  df = pd.read_csv('/content/Train_TP2_Datos_2020-2C.csv')
  df = preprocess_dataframe(df)

  features = set_up_feature_columns(df,numeric_columns,indicator_columns,bucket_columns, crossed_columns, hashed_columns)

  #Separamos el DataFrame en uno de pruebo y el de entrenamiento. TODO: Ver el de validacion
  test_lines = 0

  np.random.seed(1)
  drop_indices = np.random.choice(df.index, test_lines, replace=False)
  df_test = df.loc[drop_indices, :]
  df.drop(drop_indices, inplace=True)

  #df_validation = df.tail(200)
  #df.drop(df.tail(200).index, inplace=True)

  feature_layer = tf.keras.layers.DenseFeatures(features)
  ds = df_to_dataset(df,batch_size=56, shuffle=False)
  ds_test = df_to_dataset(df_test, batch_size=56, shuffle=False)
  #ds_validation = df_to_dataset(df_validation, batch_size=56, shuffle=False)
 
  df['Products'].value_counts()

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/

77           824
132          551
133          512
212          352
100          350
            ... 
206201         1
127117         1
69665739       1
565447         1
201209204      1
Name: Products, Length: 1175, dtype: int64

Creamos y compilamos el modelo. En esta seccion se tunean las propiedades del modelo.

In [ ]:
model = tf.keras.Sequential([
  feature_layer,
  #tf.keras.layers.Dense(3, activation='tanh'),
  #tf.keras.layers.Dense(3, activation='tanh'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

Entrenamos el modelo

In [ ]:
#Ignoren el WARNING, esta en la documentacion tambien. Nadie le da bola en StackOverflow xd.
model.fit(ds, validation_data=ds_test, epochs=110)
#model.fit(ds, epochs=60)
model.summary()

Epoch 1/110
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
175/175 [==============================] - 1s 4ms/step - loss: 0.5421 - accuracy: 0.9483
Epoch 2/110
175/175 [==============================] - 1s 4ms/step - loss: 0.4501 - accuracy: 0.9596
Epoch 3/110
175/175 [==============================] - 1s 4ms/step - loss: 0.3810 - accuracy: 0.9648
Epoch 4/110
175/175 [==============================] - 1s 4ms/step - loss: 0.3293 - accuracy: 0.9670
Epoch 5/110
175/175 [==============================] - 1s 4ms/step - loss: 0.2901 - accuracy: 0.9670
Epoch 6/110
175/175 [==============================] - 1s 4ms/step - loss: 0.2599 - accuracy: 0.9670
Epoch 7/110
175/175 [==============================] - 1s 4ms/step - loss: 0.2364 - accuracy: 0.9670
Epoch 8/110
175/175 [==============================] - 1s 4ms/step - loss: 0.2178 - accuracy: 0.9671
Epoch 9/110
175/175 [==============================] - 1s 4ms/step - loss: 0.2030 

Evaluamos el modelo

In [ ]:
#model.evaluate(ds_test)

Escribimos las predicciones

In [ ]:

frio_test_df = pd.read_csv('/content/Test_TP2_Datos_2020-2C.csv')
frio_test_df['Stage'] = 'Closed Won' #Esto esta solo para que funque todo, no lo uso. No se bien como armarlo sin los labels de Stage. TODO: Averiguar como es!
aux_df = frio_test_df[['Opportunity_ID']] #Esta columna la vuela el preprocesado sino
frio_test_df = preprocess_dataframe(frio_test_df)
frio_test_ds = df_to_dataset(frio_test_df, shuffle=False, batch_size=56)
predictions = model.predict(frio_test_ds)

aux_df.drop_duplicates(subset='Opportunity_ID', inplace=True) #Lo hacia el preprocesado pero es verdad que lo copie antes a este xd, perdon Agus, paja de dejarlo lindo.
aux_df['Target'] = predictions

#aux_df['Target'] = aux_df.groupby(by='Opportunity_ID').transform(lambda x: x.mean())
#aux_df.drop_duplicates(subset='Opportunity_ID', inplace=True)

aux_df.to_csv('prediccionesFrioFrio.csv', index=False)
'''
df = pd.read_csv('/content/Train_TP2_Datos_2020-2C.csv')
df = df[(df['Stage'] == 'Closed Won') | (df['Stage'] == 'Closed Lost')]
df = df[df['Opportunity_ID'] != 9773]
df = df[['Opportunity_ID']]
df.drop_duplicates(subset='Opportunity_ID', inplace=True)
np.random.seed(1)
drop_indices = np.random.choice(df.index, test_lines, replace=False)
df.drop(drop_indices, inplace=True)
predictions = model.predict(ds)
df['Target'] = predictions
df.to_csv('prediccionesFrioFrio.csv', index=False)
'''

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


"\ndf = pd.read_csv('/content/Train_TP2_Datos_2020-2C.csv')\ndf = df[(df['Stage'] == 'Closed Won') | (df['Stage'] == 'Closed Lost')]\ndf = df[df['Opportunity_ID'] != 9773]\ndf = df[['Opportunity_ID']]\ndf.drop_duplicates(subset='Opportunity_ID', inplace=True)\nnp.random.seed(1)\ndrop_indices = np.random.choice(df.index, test_lines, replace=False)\ndf.drop(drop_indices, inplace=True)\npredictions = model.predict(ds)\ndf['Target'] = predictions\ndf.to_csv('prediccionesFrioFrio.csv', index=False)\n"